# Model A: Sepsis Risk Classifier (XGBoost)

**Purpose**: Predict sepsis probability (0-100%) from patient vitals

**Output**: `sepsis_model.pkl` - Used by the backend to calculate risk scores

**Dataset**: Your cleaned `merged_sepsis_data.csv`

---

## Step 1: Install Required Packages

In [ ]:
# Install required packages
!pip install pandas numpy scikit-learn xgboost imbalanced-learn matplotlib seaborn --quiet
print("✅ All packages installed!")

## Step 2: Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import pickle
import os
from datetime import datetime

# ML Libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, roc_curve
)
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

print(f"✅ Libraries imported successfully!")
print(f"   Timestamp: {datetime.now()}")

## Step 3: Load YOUR Dataset

In [ ]:
# Load your cleaned merged dataset
DATA_PATH = '../merged_sepsis_data.csv'  # Adjust path if needed

print(f"Loading dataset from: {DATA_PATH}")
df = pd.read_csv(DATA_PATH)

print(f"\n✅ Dataset loaded successfully!")
print(f"   Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"   Patients: {df['Patient_ID'].nunique():,}")
print(f"\nSepsis distribution:")
print(f"   Class 0 (No Sepsis): {(df['SepsisLabel'] == 0).sum():,} ({(df['SepsisLabel'] == 0).mean()*100:.1f}%)")
print(f"   Class 1 (Sepsis):    {(df['SepsisLabel'] == 1).sum():,} ({(df['SepsisLabel'] == 1).mean()*100:.1f}%)")

In [ ]:
# Preview the data
print("Columns in dataset:")
print(list(df.columns))
print("\nFirst 5 rows:")
df.head()

In [ ]:
# Check vital sign ranges
vital_cols = ['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp']
print("Vital Sign Statistics:")
df[vital_cols].describe()

## Step 4: Feature Engineering

Your data already has MAP. We'll add ShockIndex and HR_diff.

In [ ]:
def engineer_features(df):
    """
    Create derived features for sepsis prediction.
    """
    df = df.copy()
    
    # Shock Index (normal: 0.5-0.7, elevated: >0.9)
    # Avoid division by zero
    df['ShockIndex'] = df['HR'] / df['SBP'].replace(0, np.nan)
    df['ShockIndex'] = df['ShockIndex'].fillna(0.67)  # Default normal value
    
    # Heart rate change (difference from previous reading)
    df['HR_diff'] = df.groupby('Patient_ID')['HR'].diff().fillna(0)
    
    # Clip extreme values
    df['ShockIndex'] = df['ShockIndex'].clip(0, 5)
    df['HR_diff'] = df['HR_diff'].clip(-50, 50)
    
    return df

# Apply feature engineering
print("Engineering features...")
df = engineer_features(df)

print(f"\n✅ Feature engineering complete!")
print(f"   Added: ShockIndex, HR_diff")

## Step 5: Prepare Training Data

In [ ]:
# Define feature columns (EXACT ORDER - must match backend!)
# Using columns that exist in YOUR dataset
FEATURE_COLUMNS = [
    'ICULOS',      # Hours since admission (renamed from your dataset)
    'HR',          # Heart Rate
    'O2Sat',       # SpO2
    'Temp',        # Temperature
    'SBP',         # Systolic BP
    'MAP',         # Mean Arterial Pressure (already in your data)
    'DBP',         # Diastolic BP
    'Resp',        # Respiratory Rate
    'ShockIndex',  # HR/SBP (calculated)
    'HR_diff'      # Heart Rate change (calculated)
]

TARGET_COLUMN = 'SepsisLabel'

# Verify all columns exist
missing_cols = [c for c in FEATURE_COLUMNS if c not in df.columns]
if missing_cols:
    print(f"❌ Missing columns: {missing_cols}")
else:
    print(f"✅ All feature columns found!")

# Extract features and target
X = df[FEATURE_COLUMNS].copy()
y = df[TARGET_COLUMN].copy()

print(f"\nFeature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")

In [ ]:
# Handle missing values
print("Checking for missing values...")
missing = X.isnull().sum()
missing_pct = (X.isnull().sum() / len(X)) * 100

if missing.sum() > 0:
    print(f"\nMissing values found:")
    for col in FEATURE_COLUMNS:
        if missing[col] > 0:
            print(f"   {col}: {missing[col]:,} ({missing_pct[col]:.1f}%)")
    
    # Fill with median
    X = X.fillna(X.median())
    print("\n✅ Missing values filled with median.")
else:
    print("✅ No missing values in feature columns.")

In [ ]:
# Sample data to speed up training (1.5M rows is huge)
# You can adjust SAMPLE_SIZE or set to None to use all data

SAMPLE_SIZE = 200000  # Use 200k samples for faster training

if SAMPLE_SIZE and len(X) > SAMPLE_SIZE:
    print(f"Sampling {SAMPLE_SIZE:,} rows from {len(X):,} total...")
    
    # Stratified sampling to maintain class balance
    from sklearn.model_selection import train_test_split
    X_sampled, _, y_sampled, _ = train_test_split(
        X, y, 
        train_size=SAMPLE_SIZE,
        stratify=y,
        random_state=42
    )
    X = X_sampled
    y = y_sampled
    
    print(f"✅ Sampled to {len(X):,} rows")
else:
    print(f"Using full dataset: {len(X):,} rows")

print(f"\nClass distribution after sampling:")
print(f"   Class 0: {(y == 0).sum():,} ({(y == 0).mean()*100:.1f}%)")
print(f"   Class 1: {(y == 1).sum():,} ({(y == 1).mean()*100:.1f}%)")

In [ ]:
# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    stratify=y, 
    random_state=42
)

print(f"Training set: {X_train.shape[0]:,} samples")
print(f"Test set:     {X_test.shape[0]:,} samples")

In [ ]:
# Handle class imbalance with SMOTE
print("Applying SMOTE for class balancing...")

smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

print(f"\n✅ SMOTE applied!")
print(f"   Before: {len(X_train):,} samples")
print(f"   After:  {len(X_train_balanced):,} samples")

## Step 6: Train XGBoost Model

In [ ]:
print("Training XGBoost Classifier...")
print("(This may take a few minutes)\n")

model = XGBClassifier(
    n_estimators=200,
    max_depth=6,
    min_child_weight=2,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=1.0,
    objective='binary:logistic',
    eval_metric='auc',
    use_label_encoder=False,
    random_state=42,
    n_jobs=-1
)

model.fit(
    X_train_balanced, 
    y_train_balanced,
    eval_set=[(X_test, y_test)],
    verbose=False
)

print("✅ Model training complete!")

## Step 7: Evaluate Model

In [ ]:
# Predictions
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_pred_proba)

print("="*50)
print("   MODEL PERFORMANCE METRICS")
print("="*50)
print(f"\n   Accuracy:  {accuracy:.3f}")
print(f"   Precision: {precision:.3f}")
print(f"   Recall:    {recall:.3f}  ⭐ (Catch sepsis cases!)")
print(f"   F1 Score:  {f1:.3f}")
print(f"   AUC-ROC:   {auc_roc:.3f}")
print("\n" + "="*50)

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Sepsis', 'Sepsis'],
            yticklabels=['No Sepsis', 'Sepsis'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Feature Importance
feature_importance = pd.DataFrame({
    'Feature': FEATURE_COLUMNS,
    'Importance': model.feature_importances_
}).sort_values('Importance', ascending=True)

plt.figure(figsize=(10, 6))
plt.barh(feature_importance['Feature'], feature_importance['Importance'], color='steelblue')
plt.xlabel('Feature Importance')
plt.title('XGBoost Feature Importance')
plt.show()

print("\nTop 5 Features:")
print(feature_importance.tail(5).to_string(index=False))

## Step 8: Save Model

In [ ]:
MODEL_FILENAME = 'sepsis_model.pkl'

model_package = {
    'model': model,
    'feature_columns': FEATURE_COLUMNS,
    'version': '2.0',
    'trained_at': datetime.now().isoformat(),
    'metrics': {
        'accuracy': float(accuracy),
        'precision': float(precision),
        'recall': float(recall),
        'f1': float(f1),
        'auc_roc': float(auc_roc)
    },
    'thresholds': {
        'default': 0.5,
        'high_sensitivity': 0.3,
        'high_specificity': 0.7
    }
}

with open(MODEL_FILENAME, 'wb') as f:
    pickle.dump(model_package, f)

file_size = os.path.getsize(MODEL_FILENAME) / 1024

print("="*50)
print("   MODEL SAVED!")
print("="*50)
print(f"\n   📁 File: {MODEL_FILENAME}")
print(f"   📊 Size: {file_size:.1f} KB")
print(f"   🎯 AUC-ROC: {auc_roc:.3f}")
print("\n" + "="*50)

## Step 9: Test Model

In [ ]:
def test_prediction(vitals: dict) -> dict:
    """Test model prediction."""
    with open('sepsis_model.pkl', 'rb') as f:
        pkg = pickle.load(f)
    
    model = pkg['model']
    
    sbp = vitals.get('SBP', 120)
    dbp = vitals.get('DBP', 80)
    hr = vitals.get('HR', 80)
    
    MAP = (sbp + 2 * dbp) / 3
    ShockIndex = hr / sbp if sbp > 0 else 0.67
    
    X = [[
        vitals.get('ICULOS', 1),
        hr,
        vitals.get('O2Sat', 97),
        vitals.get('Temp', 37.0),
        sbp,
        MAP,
        dbp,
        vitals.get('Resp', 18),
        ShockIndex,
        vitals.get('HR_diff', 0)
    ]]
    
    prob = model.predict_proba(X)[0][1]
    return {'risk': 'HIGH' if prob > 0.5 else 'LOW', 'probability': f"{prob*100:.1f}%"}

# Test cases
print("Testing model:\n")

normal = {'HR': 75, 'Resp': 16, 'Temp': 37.0, 'SBP': 120, 'DBP': 80, 'O2Sat': 98, 'ICULOS': 5}
print(f"Normal vitals: {test_prediction(normal)}")

warning = {'HR': 105, 'Resp': 22, 'Temp': 38.5, 'SBP': 100, 'DBP': 65, 'O2Sat': 94, 'ICULOS': 12}
print(f"Warning signs: {test_prediction(warning)}")

critical = {'HR': 125, 'Resp': 28, 'Temp': 39.2, 'SBP': 85, 'DBP': 50, 'O2Sat': 88, 'ICULOS': 24}
print(f"Critical:      {test_prediction(critical)}")

---
## ✅ Model A Complete!

**Next**: Run Model B notebook for vital forecaster.

---